# YOLO Object Detection for Image Analysis

This notebook demonstrates the use of YOLO (You Only Look Once) for object detection in images. We'll work with YOLOv8 to detect objects in various images and analyze the results.

## Table of Contents
1. Setup and Dependencies
2. Loading the YOLO Model
3. Object Detection on Sample Images
4. Visualizing Results
5. Advanced Applications

## 1. Setup and Dependencies

First, we'll install and import the necessary libraries for our object detection task.

In [ ]:
# Install ultralytics package which contains YOLO implementations
!pip install ultralytics

In [ ]:
# Import necessary libraries
from IPython.display import display, Image
from ultralytics import YOLO
import torch
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os

# Set display parameters for better visualization
%matplotlib inline
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['axes.grid'] = False

## 2. Loading the YOLO Model

Here we load YOLOv8, a state-of-the-art object detection model. YOLOv8 offers excellent performance for real-time object detection with a good balance between speed and accuracy.

In [ ]:
# Load the YOLOv8 model (nano version)
model = YOLO('yolov8n.pt')

# Print model information to verify it's loaded correctly
print(f"Model loaded: {model}")
print(f"Model task: {model.task}")

## 3. Object Detection on Sample Images

Let's start by downloading a sample image and performing object detection on it. We'll visualize both the original image and the detection results.

In [ ]:
# Download a sample image
!wget -q https://ultralytics.com/images/bus.jpg -O bus.jpg

# Display the original image
plt.figure(figsize=(10, 8))
img = cv2.imread('bus.jpg')
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # Convert BGR to RGB for display
plt.imshow(img)
plt.title('Original Image', fontsize=16)
plt.axis('off')
plt.show()

### Running Object Detection

Now we'll run the YOLO model on our image to detect objects. The model will identify various objects and provide bounding boxes, class labels, and confidence scores.

In [ ]:
# Run prediction on the sample image
results = model('bus.jpg')

# Display the results
for r in results:
    im_array = r.plot()  # plot a BGR numpy array of predictions
    im_array = cv2.cvtColor(im_array, cv2.COLOR_BGR2RGB)  # Convert BGR to RGB
    
    plt.figure(figsize=(12, 8))
    plt.imshow(im_array)
    plt.title('Object Detection Results', fontsize=16)
    plt.axis('off')
    plt.show()

### Analyzing Detection Results

Let's analyze the detection results to understand what objects were found and their confidence scores.

In [ ]:
# Analyze the detection results
result = results[0]  # Get the first result

# Extract bounding boxes, confidence scores, and class names
boxes = result.boxes
print(f"Number of objects detected: {len(boxes)}")

# Display detected objects with confidence scores
detected_objects = {}

for box in boxes:
    # Get class name
    class_id = int(box.cls.item())
    class_name = result.names[class_id]
    
    # Get confidence score
    confidence = float(box.conf.item())
    
    # Store in our dictionary
    if class_name in detected_objects:
        detected_objects[class_name].append(confidence)
    else:
        detected_objects[class_name] = [confidence]

# Print detection summary
print("\nDetection Summary:")
for obj, confidences in detected_objects.items():
    avg_conf = sum(confidences) / len(confidences)
    print(f"- {obj}: {len(confidences)} instances, Average confidence: {avg_conf:.2f}")

## 4. Testing with Multiple Images

Let's test our model on additional images to see how well it performs with different scenes and objects.

In [ ]:
# Download additional test images
!wget -q https://ultralytics.com/images/zidane.jpg -O zidane.jpg
!wget -q https://ultralytics.com/images/people.jpg -O people.jpg

# Create a function to process and display images
def process_image(image_path):
    # Run inference
    results = model(image_path)
    result = results[0]
    
    # Get original image
    img = cv2.imread(image_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    
    # Get annotated image
    annotated_img = result.plot()
    annotated_img = cv2.cvtColor(annotated_img, cv2.COLOR_BGR2RGB)
    
    # Count detected objects
    detected_counts = {}
    for box in result.boxes:
        class_id = int(box.cls.item())
        class_name = result.names[class_id]
        if class_name in detected_counts:
            detected_counts[class_name] += 1
        else:
            detected_counts[class_name] = 1
    
    # Display images side by side
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(18, 8))
    
    ax1.imshow(img)
    ax1.set_title('Original Image', fontsize=16)
    ax1.axis('off')
    
    ax2.imshow(annotated_img)
    ax2.set_title('Detection Results', fontsize=16)
    ax2.axis('off')
    
    plt.tight_layout()
    plt.show()
    
    # Print detection summary
    print(f"Detection summary for {os.path.basename(image_path)}:")
    for obj, count in detected_counts.items():
        print(f"- {obj}: {count} instances")
    print()
    
    return result

# Process each image
print("Processing image 1: zidane.jpg")
result1 = process_image('zidane.jpg')

print("\nProcessing image 2: people.jpg")
result2 = process_image('people.jpg')

## 5. Advanced Applications

### Visualizing Bounding Boxes and Confidences

Let's create a more detailed visualization showing each detected object with its bounding box coordinates and confidence score.

In [ ]:
def advanced_visualization(image_path):
    # Run detection
    results = model(image_path)
    result = results[0]
    
    # Load image
    img = cv2.imread(image_path)
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    
    # Get dimensions
    height, width = img.shape[:2]
    
    # Create figure
    fig, ax = plt.subplots(figsize=(12, 10))
    ax.imshow(img_rgb)
    
    # Colors for different classes
    np.random.seed(42)  # for consistent colors
    colors = {}
    
    # Plot each detection
    for box in result.boxes:
        # Get box coordinates
        x1, y1, x2, y2 = box.xyxy[0].tolist()
        x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)
        
        # Get class and confidence
        class_id = int(box.cls.item())
        class_name = result.names[class_id]
        confidence = float(box.conf.item())
        
        # Get color for class
        if class_name not in colors:
            colors[class_name] = np.random.rand(3,)
        color = colors[class_name]
        
        # Plot rectangle
        rect = plt.Rectangle((x1, y1), x2-x1, y2-y1, fill=False, 
                          edgecolor=color, linewidth=2)
        ax.add_patch(rect)
        
        # Add label
        label = f"{class_name}: {confidence:.2f}"
        ax.text(x1, y1-10, label, fontsize=10, bbox=dict(facecolor=color, alpha=0.5))
    
    ax.set_title(f"Advanced Visualization: {os.path.basename(image_path)}", fontsize=16)
    ax.axis('off')
    plt.tight_layout()
    plt.show()
    
    # Print detailed information
    print(f"\nDetailed detection information for {os.path.basename(image_path)}:")
    for i, box in enumerate(result.boxes):
        class_id = int(box.cls.item())
        class_name = result.names[class_id]
        confidence = float(box.conf.item())
        x1, y1, x2, y2 = box.xyxy[0].tolist()
        
        print(f"Detection {i+1}:")
        print(f"  - Class: {class_name}")
        print(f"  - Confidence: {confidence:.4f}")
        print(f"  - Bounding Box: [x1={x1:.1f}, y1={y1:.1f}, x2={x2:.1f}, y2={y2:.1f}]")
        print(f"  - Box Width: {x2-x1:.1f}, Height: {y2-y1:.1f}")
        print()

# Apply advanced visualization to our images
print("Advanced visualization for bus.jpg:")
advanced_visualization('bus.jpg')

print("\nAdvanced visualization for zidane.jpg:")
advanced_visualization('zidane.jpg')

### Custom Image Processing

Now let's create functionality to process custom images that you can upload.

In [ ]:
# Upload and process custom images
from google.colab import files

def process_custom_images():
    print("Please upload an image file:")
    uploaded = files.upload()
    
    for filename in uploaded.keys():
        print(f"\nProcessing {filename}...")
        
        # Run detection
        results = model(filename)
        result = results[0]
        
        # Display results
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(18, 8))
        
        # Original image
        img = cv2.imread(filename)
        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        ax1.imshow(img_rgb)
        ax1.set_title('Original Image', fontsize=16)
        ax1.axis('off')
        
        # Annotated image
        annotated_img = result.plot()
        annotated_img = cv2.cvtColor(annotated_img, cv2.COLOR_BGR2RGB)
        ax2.imshow(annotated_img)
        ax2.set_title('Detection Results', fontsize=16)
        ax2.axis('off')
        
        plt.tight_layout()
        plt.show()
        
        # Display detection summary
        detected_counts = {}
        for box in result.boxes:
            class_id = int(box.cls.item())
            class_name = result.names[class_id]
            confidence = float(box.conf.item())
            
            if class_name in detected_counts:
                detected_counts[class_name][0] += 1
                detected_counts[class_name][1].append(confidence)
            else:
                detected_counts[class_name] = [1, [confidence]]
        
        print(f"Detection results for {filename}:")
        for obj, (count, confidences) in detected_counts.items():
            avg_conf = sum(confidences) / len(confidences)
            print(f"- {obj}: {count} instances, Avg. confidence: {avg_conf:.2f}")

# Run the function to process custom images (uncomment to use)
# process_custom_images()

## 6. Statistical Analysis of Detections

Let's analyze the distribution of detected objects across our sample images.

In [ ]:
def statistical_analysis(image_paths):
    all_detections = {}
    image_stats = {}
    
    # Process each image
    for path in image_paths:
        results = model(path)
        result = results[0]
        image_name = os.path.basename(path)
        
        # Store detections for this image
        image_stats[image_name] = {}
        
        for box in result.boxes:
            class_id = int(box.cls.item())
            class_name = result.names[class_id]
            confidence = float(box.conf.item())
            
            # Update image stats
            if class_name in image_stats[image_name]:
                image_stats[image_name][class_name][0] += 1
                image_stats[image_name][class_name][1].append(confidence)
            else:
                image_stats[image_name][class_name] = [1, [confidence]]
                
            # Update overall stats
            if class_name in all_detections:
                all_detections[class_name] += 1
            else:
                all_detections[class_name] = 1
    
    # Generate summary statistics
    print("Object Detection Summary Across Images:")
    for image, stats in image_stats.items():
        print(f"\n{image}:")
        for obj, (count, confidences) in stats.items():
            avg_conf = sum(confidences) / len(confidences)
            print(f"  - {obj}: {count} instances, Avg. confidence: {avg_conf:.2f}")
    
    # Create visualization
    plt.figure(figsize=(12, 6))
    
    # Sort classes by number of instances
    sorted_detections = sorted(all_detections.items(), key=lambda x: x[1], reverse=True)
    classes = [item[0] for item in sorted_detections]
    counts = [item[1] for item in sorted_detections]
    
    plt.bar(classes, counts, color='skyblue')
    plt.title("Object Classes Detected Across All Images", fontsize=16)
    plt.xlabel("Object Class")
    plt.ylabel("Count")
    plt.xticks(rotation=45, ha='right')
    plt.tight_layout()
    plt.show()
    
    return all_detections, image_stats

# Perform statistical analysis on our images
image_paths = ['bus.jpg', 'zidane.jpg', 'people.jpg']
all_detections, image_stats = statistical_analysis(image_paths)

## 7. Conclusion and Next Steps

In this notebook, we've successfully implemented object detection using YOLOv8. We've demonstrated:

1. Setting up the YOLO model
2. Performing object detection on sample images
3. Analyzing and visualizing detection results
4. Processing custom images
5. Statistical analysis of detections across multiple images

### Potential Next Steps:

1. **Fine-tuning the model**: Train YOLO on your own dataset for customized object detection
2. **Video processing**: Extend the functionality to process videos instead of just images
3. **Real-time detection**: Implement webcam-based real-time object detection
4. **Specialized applications**: Focus on specific use cases like person tracking or vehicle counting
5. **Integration**: Integrate this object detection system with other applications or services

These enhancements would build upon the foundation we've established in this notebook.